In [3]:
#!pip install pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder\
.master('local[4]')\
.appName('Datasets')\
.config('spark.executor.memory','4g')\
.config('spark.driver.memory','4g')\
.getOrCreate()

sc = spark.sparkContext

### **📌list --> dataframe**

In [6]:
from pyspark.sql import Row

list_rdd = sc.parallelize([1,2,3,4,5,6,4,5]).map(lambda x: Row(x))
list_rdd.collect()

[<Row(1)>,
 <Row(2)>,
 <Row(3)>,
 <Row(4)>,
 <Row(5)>,
 <Row(6)>,
 <Row(4)>,
 <Row(5)>]

In [10]:
df_list =list_rdd.toDF(['No'])
df_list.show()

+---+
| No|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  4|
|  5|
+---+



### **📌range --> dataframe**

In [12]:
df_from_range = sc.parallelize(range(10,100,5)).map(lambda x: (x,)).toDF(["range"])
df_from_range.show(4)

+-----+
|range|
+-----+
|   10|
|   15|
|   20|
|   25|
+-----+
only showing top 4 rows



In [13]:
from pyspark.sql.types import IntegerType

df_from_range2 =spark.createDataFrame(range(10,100,5), IntegerType())
df_from_range2.show(4)

+-----+
|value|
+-----+
|   10|
|   15|
|   20|
|   25|
+-----+
only showing top 4 rows



### **📌file --> dataframe**

In [20]:
df_from_file = spark.read\
.option('sep', ';')\
.option('header', 'true')\
.option('inferSchema', 'True')\
.csv('/content/OnlineRetail.csv')

df_from_file.show(3)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 3 rows



In [21]:
df_from_file.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [22]:
df_from_file.count()

541909

In [23]:
df_from_file.select('Description', 'UnitPrice').show(5)

+--------------------+---------+
|         Description|UnitPrice|
+--------------------+---------+
|WHITE HANGING HEA...|     2,55|
| WHITE METAL LANTERN|     3,39|
|CREAM CUPID HEART...|     2,75|
|KNITTED UNION FLA...|     3,39|
|RED WOOLLY HOTTIE...|     3,39|
+--------------------+---------+
only showing top 5 rows



In [25]:
df_from_file.sort('InvoiceNo', ascending=False).show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|  C581569|    20979|36 PENCILS TUBE R...|      -5|9.12.2011 11:58|     1,25|     17315|United Kingdom|
|  C581569|    84978|HANGING HEART JAR...|      -1|9.12.2011 11:58|     1,25|     17315|United Kingdom|
|  C581568|    21258|VICTORIAN SEWING ...|      -5|9.12.2011 11:57|    10,95|     15311|United Kingdom|
|  C581499|        M|              Manual|      -1|9.12.2011 10:28|   224,69|     15498|United Kingdom|
|  C581490|    22178|VICTORIAN GLASS H...|     -12|9.12.2011 09:57|     1,95|     14397|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [26]:
df_from_file.sort('InvoiceNo', ascending=False).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [InvoiceNo#248 DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(InvoiceNo#248 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#253]
      +- FileScan csv [InvoiceNo#248,StockCode#249,Description#250,Quantity#251,InvoiceDate#252,UnitPrice#253,CustomerID#254,Country#255] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/OnlineRetail.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:string,StockCode:string,Description:string,Quantity:int,InvoiceDate:string,UnitP...




### **🔍Wordcount**

In [28]:
df = spark.read.text('/content/sample_text.txt')
df.show(truncate=50)

+--------------------------------------------------+
|                                             value|
+--------------------------------------------------+
|Eventually the centers of the galaxies will mer...|
|Despite the fireworks, this process—which is ha...|
+--------------------------------------------------+



In [33]:
from pyspark.sql.functions import explode, split, col

words = df.select(explode(split(col('value'),' ')).alias('value'))
words.show(5)

+----------+
|     value|
+----------+
|Eventually|
|       the|
|   centers|
|        of|
|       the|
+----------+
only showing top 5 rows



In [41]:
from pyspark.sql.functions import *
words.groupBy('value').count().orderBy(desc('count')).show(8)

+------+-----+
| value|count|
+------+-----+
|   the|   25|
|    of|   12|
|     a|    8|
|  will|    7|
|   and|    7|
|    in|    7|
|galaxy|    5|
| stars|    4|
+------+-----+
only showing top 8 rows



In [42]:
#alternative way
words.groupBy("value").agg({"value": "count"}).orderBy(desc("count(value)")).show(10)

+------+------------+
| value|count(value)|
+------+------------+
|   the|          25|
|    of|          12|
|     a|           8|
|    in|           7|
|   and|           7|
|  will|           7|
|galaxy|           5|
| stars|           4|
|    is|           4|
| black|           4|
+------+------------+
only showing top 10 rows



### **🔍SQL Query**

In [43]:
df_sql = spark.read\
.option('sep', ';')\
.option('header', 'true')\
.option('inferSchema', 'True')\
.csv('/content/OnlineRetail.csv')


+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 3 rows



In [55]:
df_sql.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: string, CustomerID: int, Country: string]

In [56]:
df_sql.show(3)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 3 rows



In [57]:
df_sql.createOrReplaceTempView('table')

In [58]:
spark.sql("select * from table").show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|1.12.2010 08:26|     7,65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|1.12.2010 08:

In [60]:
spark.sql("select Country, Sum(UnitPrice) UnitPrice from table Group By Country order by UnitPrice desc").show(10)

+--------------+---------+
|       Country|UnitPrice|
+--------------+---------+
|United Kingdom|  94911.0|
|          EIRE|   9423.0|
|       Germany|   7930.0|
|        France|   6288.0|
|         Spain|   2927.0|
|       Finland|   1578.0|
|       Belgium|   1503.0|
|        Norway|   1451.0|
|   Switzerland|   1267.0|
|        Sweden|    921.0|
+--------------+---------+
only showing top 10 rows



## 🔍String Operations

In [65]:
simple_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("/content/simple_dirty_data.csv")

In [68]:
simple_df.show(5)

+------+-------+---+--------+-----------+-----------+-----------+---------------+
|sirano|   isim|yas|cinsiyet|     meslek|      sehir|aylik_gelir|       mal_mulk|
+------+-------+---+--------+-----------+-----------+-----------+---------------+
|     1|  Cemal| 35|       E|       Isci|     Ankara|     3500.0|          araba|
|     2| ceyda | 42|       K|      Memur|    Kayseri|     4200.0|       araba|ev|
|     3|  Timur| 30|    null|   Müzüsyen|Istanbul   |     9000.0|araba|ev|yazlık|
|     4| Burcu | 29|       K|Pazarlamacı|     Ankara|     4200.0|          araba|
|     5|Yasemin| 23|       K|Pazarlamaci|      Bursa|     4800.0|          araba|
+------+-------+---+--------+-----------+-----------+-----------+---------------+
only showing top 5 rows



*📌Concat()*

In [73]:
df_concat = simple_df.withColumn("meslek_sehir", concat('meslek', lit('&'), 'sehir'))
df_concat.show(5)

+------+-------+---+--------+-----------+-----------+-----------+---------------+--------------------+
|sirano|   isim|yas|cinsiyet|     meslek|      sehir|aylik_gelir|       mal_mulk|        meslek_sehir|
+------+-------+---+--------+-----------+-----------+-----------+---------------+--------------------+
|     1|  Cemal| 35|       E|       Isci|     Ankara|     3500.0|          araba|         Isci&Ankara|
|     2| ceyda | 42|       K|      Memur|    Kayseri|     4200.0|       araba|ev|       Memur&Kayseri|
|     3|  Timur| 30|    null|   Müzüsyen|Istanbul   |     9000.0|araba|ev|yazlık|Müzüsyen&Istanbul   |
|     4| Burcu | 29|       K|Pazarlamacı|     Ankara|     4200.0|          araba|Pazarlamacı&    A...|
|     5|Yasemin| 23|       K|Pazarlamaci|      Bursa|     4800.0|          araba|   Pazarlamaci&Bursa|
+------+-------+---+--------+-----------+-----------+-----------+---------------+--------------------+
only showing top 5 rows



*📌format_number()*

In [75]:
df_number_format = simple_df.withColumn("aylik_gelir_format", format_number("aylik_gelir",3))
df_number_format.show(n=5, truncate=False)

+------+-------+---+--------+-----------+-----------+-----------+---------------+------------------+
|sirano|isim   |yas|cinsiyet|meslek     |sehir      |aylik_gelir|mal_mulk       |aylik_gelir_format|
+------+-------+---+--------+-----------+-----------+-----------+---------------+------------------+
|1     |Cemal  |35 |E       |Isci       |Ankara     |3500.0     |araba          |3,500.000         |
|2     |ceyda  |42 |K       |Memur      |Kayseri    |4200.0     |araba|ev       |4,200.000         |
|3     |Timur  |30 |null    |Müzüsyen   |Istanbul   |9000.0     |araba|ev|yazlık|9,000.000         |
|4     |Burcu  |29 |K       |Pazarlamacı|    Ankara |4200.0     |araba          |4,200.000         |
|5     |Yasemin|23 |K       |Pazarlamaci|Bursa      |4800.0     |araba          |4,800.000         |
+------+-------+---+--------+-----------+-----------+-----------+---------------+------------------+
only showing top 5 rows



*📌 lower(), initcap(), length()*

In [76]:
df_lower = simple_df \
.withColumn("meslek_lower", lower(col("meslek"))) \
.withColumn("isim_initcap", initcap(col("isim"))) \
.withColumn("sehir_length", length(col("sehir")))

df_lower.show(n=5, truncate=False)

+------+-------+---+--------+-----------+-----------+-----------+---------------+------------+------------+------------+
|sirano|isim   |yas|cinsiyet|meslek     |sehir      |aylik_gelir|mal_mulk       |meslek_lower|isim_initcap|sehir_length|
+------+-------+---+--------+-----------+-----------+-----------+---------------+------------+------------+------------+
|1     |Cemal  |35 |E       |Isci       |Ankara     |3500.0     |araba          |isci        |Cemal       |6           |
|2     |ceyda  |42 |K       |Memur      |Kayseri    |4200.0     |araba|ev       |memur       |Ceyda       |7           |
|3     |Timur  |30 |null    |Müzüsyen   |Istanbul   |9000.0     |araba|ev|yazlık|müzüsyen    |Timur       |11          |
|4     |Burcu  |29 |K       |Pazarlamacı|    Ankara |4200.0     |araba          |pazarlamacı |Burcu       |10          |
|5     |Yasemin|23 |K       |Pazarlamaci|Bursa      |4800.0     |araba          |pazarlamaci |Yasemin     |5           |
+------+-------+---+--------+---

*📌 trim()*

In [77]:
df_trim = simple_df \
.withColumn("sehir_rtrim", rtrim(col("sehir"))) \
.withColumn("sehir_ltrim", ltrim(col("sehir"))) \
.withColumn("sehir_trim", trim(col("sehir")))

df_trim.show(n=5, truncate=False)

+------+-------+---+--------+-----------+-----------+-----------+---------------+-----------+-----------+----------+
|sirano|isim   |yas|cinsiyet|meslek     |sehir      |aylik_gelir|mal_mulk       |sehir_rtrim|sehir_ltrim|sehir_trim|
+------+-------+---+--------+-----------+-----------+-----------+---------------+-----------+-----------+----------+
|1     |Cemal  |35 |E       |Isci       |Ankara     |3500.0     |araba          |Ankara     |Ankara     |Ankara    |
|2     |ceyda  |42 |K       |Memur      |Kayseri    |4200.0     |araba|ev       |Kayseri    |Kayseri    |Kayseri   |
|3     |Timur  |30 |null    |Müzüsyen   |Istanbul   |9000.0     |araba|ev|yazlık|Istanbul   |Istanbul   |Istanbul  |
|4     |Burcu  |29 |K       |Pazarlamacı|    Ankara |4200.0     |araba          |    Ankara |Ankara     |Ankara    |
|5     |Yasemin|23 |K       |Pazarlamaci|Bursa      |4800.0     |araba          |Bursa      |Bursa      |Bursa     |
+------+-------+---+--------+-----------+-----------+-----------

*📌 replace(), split()*

In [79]:
df_replace = simple_df \
.withColumn("sehir_ist", regexp_replace("sehir", "Ist", "İST")) \
.withColumn("mal_mulk_split", split("mal_mulk", "\\|")) \
.withColumn("mal_mulk_ilk_eleman", col("mal_mulk_split")[0])

df_replace.show(n=5, truncate=False)

+------+-------+---+--------+-----------+-----------+-----------+---------------+-----------+-------------------+-------------------+
|sirano|isim   |yas|cinsiyet|meslek     |sehir      |aylik_gelir|mal_mulk       |sehir_ist  |mal_mulk_split     |mal_mulk_ilk_eleman|
+------+-------+---+--------+-----------+-----------+-----------+---------------+-----------+-------------------+-------------------+
|1     |Cemal  |35 |E       |Isci       |Ankara     |3500.0     |araba          |Ankara     |[araba]            |araba              |
|2     |ceyda  |42 |K       |Memur      |Kayseri    |4200.0     |araba|ev       |Kayseri    |[araba, ev]        |araba              |
|3     |Timur  |30 |null    |Müzüsyen   |Istanbul   |9000.0     |araba|ev|yazlık|İSTanbul   |[araba, ev, yazlık]|araba              |
|4     |Burcu  |29 |K       |Pazarlamacı|    Ankara |4200.0     |araba          |    Ankara |[araba]            |araba              |
|5     |Yasemin|23 |K       |Pazarlamaci|Bursa      |4800.0   

In [80]:
df_replace.printSchema()

root
 |-- sirano: integer (nullable = true)
 |-- isim: string (nullable = true)
 |-- yas: integer (nullable = true)
 |-- cinsiyet: string (nullable = true)
 |-- meslek: string (nullable = true)
 |-- sehir: string (nullable = true)
 |-- aylik_gelir: double (nullable = true)
 |-- mal_mulk: string (nullable = true)
 |-- sehir_ist: string (nullable = true)
 |-- mal_mulk_split: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- mal_mulk_ilk_eleman: string (nullable = true)



## *🔍Save DataFrame*

In [82]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("/content/simple_dirty_data.csv")

In [83]:
df.show(10)

+------+--------+---+--------+-----------+-----------+-----------+--------------------+
|sirano|    isim|yas|cinsiyet|     meslek|      sehir|aylik_gelir|            mal_mulk|
+------+--------+---+--------+-----------+-----------+-----------+--------------------+
|     1|   Cemal| 35|       E|       Isci|     Ankara|     3500.0|               araba|
|     2|  ceyda | 42|       K|      Memur|    Kayseri|     4200.0|            araba|ev|
|     3|   Timur| 30|    null|   Müzüsyen|Istanbul   |     9000.0|     araba|ev|yazlık|
|     4|  Burcu | 29|       K|Pazarlamacı|     Ankara|     4200.0|               araba|
|     5| Yasemin| 23|       K|Pazarlamaci|      Bursa|     4800.0|               araba|
|     6|     Ali| 33|       E|      Memur|     Ankara|     4250.0|                  ev|
|     7|   Dilek| 29|       K|Pazarlamaci|   Istanbul|     7300.0|        araba|yazlık|
|     8|   Murat| 31|       E|   Müzüsyen|   Istanbul|    12000.0|araba|ev|dükkan|y...|
|     9|   Ahmet| 33|       E|  

In [87]:
from pyspark.sql import functions as F
df2 = df \
.withColumn("isim", F.trim(F.initcap(df.isim))) \
.withColumn("cinsiyet", F.when(df['cinsiyet'].isNull(), "Unknown").otherwise(df['cinsiyet'])) \
.withColumn("sehir", F.when(df['sehir'].isNull(), "UNKNOWN").otherwise(F.trim(F.upper(df['sehir']))))


df2.show(15)

+------+--------+---+--------+-----------+--------+-----------+--------------------+
|sirano|    isim|yas|cinsiyet|     meslek|   sehir|aylik_gelir|            mal_mulk|
+------+--------+---+--------+-----------+--------+-----------+--------------------+
|     1|   Cemal| 35|       E|       Isci|  ANKARA|     3500.0|               araba|
|     2|   Ceyda| 42|       K|      Memur| KAYSERI|     4200.0|            araba|ev|
|     3|   Timur| 30| Unknown|   Müzüsyen|ISTANBUL|     9000.0|     araba|ev|yazlık|
|     4|   Burcu| 29|       K|Pazarlamacı|  ANKARA|     4200.0|               araba|
|     5| Yasemin| 23|       K|Pazarlamaci|   BURSA|     4800.0|               araba|
|     6|     Ali| 33|       E|      Memur|  ANKARA|     4250.0|                  ev|
|     7|   Dilek| 29|       K|Pazarlamaci|ISTANBUL|     7300.0|        araba|yazlık|
|     8|   Murat| 31|       E|   Müzüsyen|ISTANBUL|    12000.0|araba|ev|dükkan|y...|
|     9|   Ahmet| 33|       E|     Doktor|  ANKARA|   180000.0|  

In [ ]:
#coalesce()dagitik haldeki verileri birlestiriyor.
df2.coalesce(1) \
.write \
.mode("overwrite") \
.option("sep",",") \
.option("header","True") \
.csv("Desktop/Udemy-ApacheSparkCourse/dataset")

In [91]:
df3 = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("Desktop/Udemy-ApacheSparkCourse/dataset")

df3.show(15)

+------+--------+---+--------+-----------+--------+-----------+--------------------+
|sirano|    isim|yas|cinsiyet|     meslek|   sehir|aylik_gelir|            mal_mulk|
+------+--------+---+--------+-----------+--------+-----------+--------------------+
|     1|   Cemal| 35|       E|       Isci|  ANKARA|     3500.0|               araba|
|     2|   Ceyda| 42|       K|      Memur| KAYSERI|     4200.0|            araba|ev|
|     3|   Timur| 30| Unknown|   Müzüsyen|ISTANBUL|     9000.0|     araba|ev|yazlık|
|     4|   Burcu| 29|       K|Pazarlamacı|  ANKARA|     4200.0|               araba|
|     5| Yasemin| 23|       K|Pazarlamaci|   BURSA|     4800.0|               araba|
|     6|     Ali| 33|       E|      Memur|  ANKARA|     4250.0|                  ev|
|     7|   Dilek| 29|       K|Pazarlamaci|ISTANBUL|     7300.0|        araba|yazlık|
|     8|   Murat| 31|       E|   Müzüsyen|ISTANBUL|    12000.0|araba|ev|dükkan|y...|
|     9|   Ahmet| 33|       E|     Doktor|  ANKARA|   180000.0|  

## *🔍Manual Schema*

In [93]:
df_schema = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",";") \
.csv("/content/OnlineRetail.csv")

In [102]:
df_schema.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [95]:
df_schema.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [96]:
#UnitPrice sutununun type uygun olmadigi icin manuel schema yapayagim.

from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType, DateType,DoubleType

manual_schema = StructType([
    StructField("InvoiceNo", StringType(), True),
    StructField("StockCode", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("InvoiceDate", StringType(), True),
    StructField("UnitPrice", FloatType(), True),
    StructField("CustomerID", IntegerType(), True),
    StructField("Country", StringType(), True)

])

In [97]:
df_schema_1 = spark.read \
.option("header","True") \
.option("sep",";") \
.schema(manual_schema) \
.csv("/content/OnlineRetail.csv")

In [100]:
df_schema_1.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     null|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     null|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     null|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     null|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     null|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [99]:
df_schema_1.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: float (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [103]:
#yukaridaki islem sonucu UnitPrice uygun formata donusmedi,Sebebi virgullerle ayrilmis olmasi olabilir.
df_schema = df_schema.withColumn("UnitPrice",F.regexp_replace(F.col("UnitPrice"), ",","."))

In [104]:
df_schema.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [110]:
df_schema \
    .coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("sep",";") \
    .option("header","true") \
    .csv("/content/onlineretail.csv")

In [111]:
df_schema_1 = spark.read \
.option("header","True") \
.option("sep",";") \
.schema(manual_schema) \
.csv("/content/onlineretail.csv/part-00000-ea16faa7-2a44-4e63-a6b0-87e650e8f8bb-c000.csv")

In [112]:
df_schema_1.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [113]:
df_schema_1.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: float (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



## *🔍Time Operations*

In [130]:
df_time = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",";") \
.csv("/content/OnlineRetail.csv") \
.select("InvoiceDate").distinct()

In [131]:
df_time.show(10)

+----------------+
|     InvoiceDate|
+----------------+
| 3.12.2010 16:50|
| 7.12.2010 12:28|
| 8.12.2010 15:02|
|10.12.2010 09:53|
|12.12.2010 13:32|
|15.12.2010 13:21|
|16.12.2010 08:41|
|17.12.2010 09:52|
| 9.01.2011 11:43|
|11.01.2011 11:38|
+----------------+
only showing top 10 rows



In [132]:
df_time.printSchema()

root
 |-- InvoiceDate: string (nullable = true)



In [133]:
time_format = 'dd.MM.yyyy HH:mm'

In [ ]:
from pyspark.sql import functions as F

df_time_format1 = df_time.withColumn("InvoiceDate", F.trim(F.col("InvoiceDate"))) \
.withColumn("normal_tarih", F.to_date(F.col("InvoiceDate"), time_format)) \
.withColumn("standart_ts", F.to_timestamp(F.col("InvoiceDate"), time_format)) \

df_time_format1.show(10)

In [119]:
df2.printSchema()

root
 |-- sirano: integer (nullable = true)
 |-- isim: string (nullable = true)
 |-- yas: integer (nullable = true)
 |-- cinsiyet: string (nullable = true)
 |-- meslek: string (nullable = true)
 |-- sehir: string (nullable = true)
 |-- aylik_gelir: double (nullable = true)
 |-- mal_mulk: string (nullable = true)



In [ ]:
format_tr = "dd/MM/yyyy HH:mm:ss"
format_eng = "MM-dd-yyyy HH:mm:ss"

df_time_format2 = df_time_format1 \
.withColumn("TSTR", F.date_format(F.col("standart_ts"), format_tr)) \
.withColumn("TSENG", F.date_format(F.col("standart_ts"), format_eng)) \
.withColumn("unix_time", F.unix_timestamp(F.col("standart_ts"))) \

df_time_format2.show(10)

In [ ]:
df_time_format2.printSchema()

In [ ]:
df_time_format3 = df_time_format2\
.withColumn("bir_yil", F.date_add(F.col("standart_ts"), 365)) \
.withColumn("yil", F.year(F.col("standart_ts"))) \
.withColumn("fark", F.datediff(F.col("bir_yil"), F.col("standart_ts")))


df_time_format3.show(10)